In [22]:
## all imports
from IPython.display import HTML
import numpy as np
import urllib
from bs4 import BeautifulSoup#this is beautiful soup
import time
import operator
import socket
import _pickle as cPickle
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")
import json
import requests
from lxml import html
import datetime
from collections import OrderedDict
import argparse
from secrets import *

In [42]:
import json
import requests
from lxml import html
from collections import OrderedDict
import argparse

def parse(source,destination,date):
	for i in range(5):
		try:
			url = "https://www.expedia.com/Flights-Search?trip=oneway&leg1=from:"+source+",to:"+destination+",departure:"+date+"TANYT&passengers=adults:1,children:0,seniors:0,infantinlap:Y&options=cabinclass%3Aeconomy&mode=search&origref=www.expedia.com"
			response = requests.get(url)
			parser = html.fromstring(response.text)
			json_data_xpath = parser.xpath("//script[@id='cachedResultsJson']//text()")
			if(len(json_data_xpath)==0):
				break
			raw_json =json.loads(json_data_xpath[0])
			flight_data = json.loads(raw_json["content"])

			flight_info  = OrderedDict() 
			lists=[]

			for i in flight_data['legs'].keys():
				total_distance =  flight_data['legs'][i]["formattedDistance"]
				exact_price = flight_data['legs'][i]['price']['totalPriceAsDecimal']

				departure_location_city = flight_data['legs'][i]['departureLocation']['airportCity']
				departure_location_airport_code = flight_data['legs'][i]['departureLocation']['airportCode']
				
				arrival_location_airport_code = flight_data['legs'][i]['arrivalLocation']['airportCode']
				arrival_location_city = flight_data['legs'][i]['arrivalLocation']['airportCity']
				airline_name = flight_data['legs'][i]['carrierSummary']['airlineName']
				
				no_of_stops = flight_data['legs'][i]["stops"]
				flight_duration = flight_data['legs'][i]['duration']
				flight_hour = flight_duration['hours']
				flight_minutes = flight_duration['minutes']
				flight_days = flight_duration['numOfDays']

				if no_of_stops==0:
					stop = "Nonstop"
				else:
					stop = str(no_of_stops)+' Stop'

				total_flight_duration = "{0} days {1} hours {2} minutes".format(flight_days,flight_hour,flight_minutes)
				if 'timeline' in flight_data['legs'][i].keys():
					if flight_data['legs'][i]['timeline'] is not None:
						carrier = flight_data['legs'][i]['timeline'][0]['carrier']
						plane = carrier['plane']
						plane_code = carrier['planeCode']
						flight_number=carrier['flightNumber']
					else:
						continue
				else:
					continue
				formatted_price = "{0:.2f}".format(exact_price)

				if not airline_name:
					airline_name = carrier['operatedBy']
				
				#timings = []
				for timeline in  flight_data['legs'][i]['timeline']:
					if 'departureAirport' in timeline.keys():
						departure_airport = timeline['departureAirport']['longName']
						departure_time = timeline['departureTime']['time']
						arrival_airport = timeline['arrivalAirport']['longName']
						arrival_time = timeline['arrivalTime']['time']
						flight_timing = {
											'departure_airport':departure_airport,
											'departure_time':departure_time,
											'arrival_airport':arrival_airport,
											'arrival_time':arrival_time
						}
						#timings.append(flight_timing)

				flight_info={'stops':stop,
					'ticket price':formatted_price,
					#'departure':departure,
					#'arrival':arrival,
					'flight duration':total_flight_duration,
					'airline':airline_name,
					'plane':plane,
					'timings':flight_timing,
					'plane code':plane_code,
					'flight_number':flight_number         
				}
				lists.append(flight_info)
			sortedlist = sorted(lists, key=lambda k: k['ticket price'],reverse=False)
			return sortedlist
		
		except ValueError:
			print("Rerying...")
			
		return {"error":"failed to process the page",}

#airport=pd.read_csv('airports1.csv')
#airport=airport.dropna(subset=['type','iata_code'])
#airport=airport[(airport['iata_code']!='DEL')|(airport['iata_code']!='BTR')]
#airport=airport[((airport['iso_country']!='IN') & (airport['type']=='large_airport')) | ((airport['iso_country']=='IN')&((airport['type']!='heliport')|(airport['type']!='closed')))]
#airport=airport.reset_index()
#airport=airport['iata_code']
#a=set(airport)
#a=list(a)
#airport=pd.DataFrame(a)
airport=pd.read_csv('airports.csv')
source = 'DEL'
print(airport)
flights=[]
number_of_flights=0
for index,row in airport.iterrows():
    print(index)
    destination = row[1]
    date = '12/20/2017'
    print("Fetching flight details")
    scraped_data = parse(source,destination,date)
    df=pd.DataFrame(scraped_data)
    if df.size == 0:
        continue
    for ind,r in df.iterrows():
        t1=df['timings'][ind]['departure_time']
        t1=datetime.datetime.strptime(t1,'%I:%M%p')
        time_lower=datetime.datetime.strptime('12:00pm','%I:%M%p')
        time_upper=datetime.datetime.strptime('6:00pm','%I:%M%p')
        if t1.time()<=time_upper.time() and t1.time()>=time_lower.time():
            number_of_flights=number_of_flights + 1
            print('Accepted')
        else:
            print('Nope')    
airport=pd.read_csv('airports_arrive.csv')
destination = 'DEL'
print(airport)
for index,row in airport.iterrows():
    print(index)
    source = row[1]
    date = '12/20/2017'
    print("Fetching flight details")
    scraped_data = parse(source,destination,date)
    df=pd.DataFrame(scraped_data)
    if df.size == 0:
        continue
    for ind,r in df.iterrows():
        t1=df['timings'][ind]['arrival_time']
        t1=datetime.datetime.strptime(t1,'%I:%M%p')
        time_lower=datetime.datetime.strptime('12:00pm','%I:%M%p')
        time_upper=datetime.datetime.strptime('6:00pm','%I:%M%p')
        if t1.time()<=time_upper.time() and t1.time()>=time_lower.time():
            number_of_flights=number_of_flights + 1
            print('Accepted')
        else:
            print('Nope')
print('Total number of flghts:',number_of_flights)

    Unnamed: 0    0
0            4  JLR
1           10  GAY
2           30  MEL
3           37  HYD
4           42  IXC
5           50  DOH
6           52  HKG
7           58  IDR
8           65  MAA
9           96  AUH
10         106  ATQ
11         126  CAN
12         147  PUT
13         175  GAU
14         179  AMD
15         182  RMD
16         196  RPR
17         198  BKB
18         209  MCT
19         231  BBI
20         234  PNQ
21         240  DBD
22         249  PNY
23         253  SHA
24         261  PAT
25         268  AMS
26         271  VGA
27         279  SIN
28         283  BEK
29         288  ADW
..         ...  ...
43         444  YVR
44         451  RTC
45         455  IXB
46         474  NRT
47         486  KUU
48         493  CCU
49         498  KUL
50         525  BDQ
51         532  VTZ
52         534  KMG
53         537  JED
54         545  IXJ
55         546  DHM
56         563  IXD
57         567  BOM
58         572  VNS
59         588  ADD
60         607  IXL


KeyboardInterrupt: 

In [118]:
airport=pd.read_csv('airports1.csv')
airport=airport.dropna(subset=['type','iata_code'])
airport=airport[(airport['iata_code']!='DEL')]
airport=airport[((airport['iso_country']!='IN') & (airport['type']=='large_airport')) | ((airport['iso_country']=='IN')&((airport['type']!='heliport')|(airport['type']!='closed')))]
airport=airport.reset_index()
airport=airport['iata_code']
a=set(airport)
a=list(a)
airport=pd.DataFrame(a)
airport.to_csv('air.csv')

In [ ]:
print('BLR' in airport)